In [ ]:
import h5py
import os
import re

import numpy as np

In [ ]:
HOME_PATH = '/home/shubhams/Hercules/kidstube-data/'
SECONDS_PER_CLIP = 3
READ_DATASET_PATH = 'processed/aggregate_{0}_sec/unbalanced_data.hdf5'.format(SECONDS_PER_CLIP)
WRITE_DATASET_PATH = 'processed/aggregate_{0}_sec/balanced_data.hdf5'.format(SECONDS_PER_CLIP)
label_ctr = [12313, 6795, 2268, 3244]

In [ ]:
def read_hdf5(name, path=HOME_PATH + READ_DATASET_PATH):
    f = h5py.File(path, 'r')
    return f[name][()]

In [ ]:
frames = read_hdf5('frames')
annotations = read_hdf5('annotations')

In [ ]:
label_check_ctr = [0, 0, 0, 0]
for i in range(annotations.shape[0]):
    label_check_ctr[int(annotations[i])] += 1
print(label_check_ctr)

In [ ]:
def save_checkpoint(frame_data, annotations, path=HOME_PATH+WRITE_DATASET_PATH):
    with h5py.File(path, 'a', libver='latest') as f:
        frame_data = np.array(frame_data)
        annotations = np.array(annotations)
        
        try:
            frame_dset = f['frames']
            annotation_dset = f['annotations']
        except KeyError:
            frame_dset = f.create_dataset('frames', shape=(0, 6*SECONDS_PER_CLIP, 512), maxshape=(None, 6*SECONDS_PER_CLIP, 512), compression = 'gzip')
            annotation_dset = f.create_dataset('annotations', shape=(0, ), maxshape=(None,), compression = 'gzip')
            f.swmr_mode = True

        new_frame_shape = frame_data.shape[0]
        new_annotation_shape = annotations.shape[0]
        
        frame_dset.resize(frame_dset.shape[0] + new_frame_shape, axis=0)
        annotation_dset.resize(annotation_dset.shape[0] + new_annotation_shape, axis=0)
        
        frame_dset[-new_frame_shape:] = frame_data
        annotation_dset[-new_annotation_shape:] = annotations
        print(frame_dset.shape)

In [ ]:
annotated_frames = []
annotation_labels = []

print(np.sum(label_ctr))
save_every = 2000
ctr = 0

rand_indices = np.random.permutation(frames.shape[0])
for rand_idx in rand_indices:
    if not np.any(label_ctr):
        break
    label = int(annotations[rand_idx])
    if label_ctr[label] > 0:     
        ctr += 1
        annotated_frames.append(frames[rand_idx])
        annotation_labels.append(annotations[rand_idx])
        label_ctr[label] = label_ctr[label] - 1
        if ctr % save_every == 0:
            save_checkpoint(annotated_frames, annotation_labels)
            print(label_ctr)
            annotated_frames = []
            annotation_labels = []
save_checkpoint(annotated_frames, annotation_labels)

In [ ]:
balanced_annotations = read_hdf5('annotations', HOME_PATH+WRITE_DATASET_PATH)

label_check_ctr = [0, 0, 0, 0]
for i in range(balanced_annotations.shape[0]):
    label_check_ctr[int(balanced_annotations[i])] += 1
print(label_check_ctr)